In [1]:
import tensorflow as tf

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ReduceLROnPlateau
import pickle
import time

# model_name = "ConvNet-64x2-{}".format(int(time.time()))


x = pickle.load(open('X.pickle',"rb"))
y = pickle.load(open('y.pickle',"rb"))


In [3]:
x = x/255.0

In [4]:
import time

dense_layers = [0,1,2]
layer_sizes = [32,64,128]
conv_layers = [1,2,3]


In [ ]:
for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer,layer_size,dense_layer,int(time.time()))
            tensorboard = TensorBoard(log_dir="./optimizing models/{}".format(NAME))
            # print(NAME)
            
            model = Sequential()

            model.add(Conv2D(layer_size, (3,3), input_shape = x.shape[1:]))
            model.add(Activation("relu"))
            model.add(MaxPool2D(pool_size=(2,2)))
            
            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3,3), input_shape = x.shape[1:]))
                model.add(Activation("relu"))
                model.add(MaxPool2D(pool_size=(2,2)))
            
            for l in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation("relu"))
                
            model.add(Flatten())

            # model.add(Dense(128))
            # model.add(Activation("tanh"))

            model.add(Dense(1))
            model.add(Activation(activation="sigmoid"))
            model.compile(optimizer="adam",loss="binary_crossentropy",metrics = ["accuracy"])
            model.fit(x,y,batch_size=64,validation_split=0.1,callbacks=[tensorboard],epochs=2)

Train on 9000 samples, validate on 1000 samples
Epoch 1/2
9000/9000 [==============================] - 22s 2ms/step - loss: 0.6664 - acc: 0.5896 - val_loss: 0.6353 - val_acc: 0.6380
Epoch 2/2
9000/9000 [==============================] - 22s 2ms/step - loss: 0.6163 - acc: 0.6714 - val_loss: 0.6066 - val_acc: 0.6720
Train on 9000 samples, validate on 1000 samples
Epoch 1/2
9000/9000 [==============================] - 202s 22ms/step - loss: 0.6732 - acc: 0.5826 - val_loss: 0.6337 - val_acc: 0.6350
Epoch 2/2
9000/9000 [==============================] - 215s 24ms/step - loss: 0.6021 - acc: 0.6791 - val_loss: 0.5916 - val_acc: 0.6750
Train on 9000 samples, validate on 1000 samples
Epoch 1/2
9000/9000 [==============================] - 366s 41ms/step - loss: 0.6880 - acc: 0.5343 - val_loss: 0.6900 - val_acc: 0.5160
Epoch 2/2
9000/9000 [==============================] - 386s 43ms/step - loss: 0.6513 - acc: 0.6224 - val_loss: 0.6257 - val_acc: 0.6560
Train on 9000 samples, validate on 1000 samp

In [88]:
model.save("Conv.model")

In [28]:
conv = tf.keras.models.load_model("Conv.model")

In [29]:
conv.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 48, 48, 64)        640       
_________________________________________________________________
activation_4 (Activation)    (None, 48, 48, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 22, 22, 64)        36928     
_________________________________________________________________
activation_5 (Activation)    (None, 22, 22, 64)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 11, 11, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 7744)              0         
__________

In [30]:
pred = model.predict(x)

In [31]:
pred.shape

(10000, 1)

In [32]:
import numpy as np

In [33]:
predictions = np.zeros((x.shape[0],1))
for i in range(pred.shape[0]):
    predictions[i,0] = 1 if pred[i,0]>0.5 else 0

In [34]:
true_labels = y.reshape(y.shape[0],1)

In [35]:
accuracy = 100 - np.mean(np.abs(predictions-true_labels))*100

In [36]:
print("The accuracy of the model is: %.2f"%(accuracy))

The accuracy of the model is: 68.68
